# GitHub Stats for FLORIS
This notebook is used to connect to the GitHub stats database for the FLORIS repo to extract and visualize data



In [1]:
# add path above this notebook
import sys
sys.path.append("..") 
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dateutil

In [2]:
# import database
import database as dbc

/Users/pfleming/Projects/ghstats/notebooks/../database.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (repo_code is None) or (repo_code is'wisdem_floris'):
/Users/pfleming/Projects/ghstats/notebooks/../database.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'nrel_floris':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'wfc_tools':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (repo_code is None) or (repo_code is'wisdem_floris'):
/Users/pfleming/Projects/ghstats/notebooks/../database.py:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'nrel_floris':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'wfc_tools':
/Users/pfleming/Projects/ghstats/notebooks/.

In [3]:
# connect to the database

db = dbc.Database()

/Users/pfleming/Projects/ghstats/notebooks/../database.py:42: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (repo_code is None) or (repo_code is'wisdem_floris'):
/Users/pfleming/Projects/ghstats/notebooks/../database.py:44: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'nrel_floris':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:46: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'wfc_tools':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:67: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (repo_code is None) or (repo_code is'wisdem_floris'):
/Users/pfleming/Projects/ghstats/notebooks/../database.py:69: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'nrel_floris':
/Users/pfleming/Projects/ghstats/notebooks/../database.py:71: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif repo_code is 'wfc_tools':
/Users/pfleming/Projects/ghstats/notebooks/.

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres

In [ ]:
# retrieve data in the following range
# timestamps in format YYYY-MM-DD
from_date = '2018-01-01'
to_date = datetime.datetime.today().strftime('%Y-%m-%d') # set to today


In [ ]:
print("FROM DATE: " + from_date)
print("TO DATE: " + to_date)

In [ ]:
results_wisdem = db.getResults(from_date, to_date, repo_code='wisdem_floris')
results_nrel_floris = db.getResults(from_date, to_date, repo_code='nrel_floris')
results_wfc_tools = db.getResults(from_date, to_date, repo_code='wfc_tools')

In [ ]:
# Mix together
df_1 = pd.DataFrame(results_wisdem)
df_1['repo'] = 'floris_wisdem'
df_2 = pd.DataFrame(results_nrel_floris)
df_2['repo'] = 'floris_nrel'
df_3 = pd.DataFrame(results_wfc_tools)
df_3['repo'] = 'wfc_tools'

df = df_1.append(df_2).append(df_3)
df['timestamp'] = pd.to_datetime(df.timestamp)

In [ ]:
# Mix together with 1 week resampling
df_1 = pd.DataFrame(results_wisdem)
df_1['timestamp'] = pd.to_datetime(df_1.timestamp)
df_1 = df_1.resample('7D', on='timestamp').sum().reset_index()
df_1['repo'] = 'floris_wisdem'

df_2 = pd.DataFrame(results_nrel_floris)
df_2['timestamp'] = pd.to_datetime(df_2.timestamp)
df_2 = df_2.resample('7D', on='timestamp').sum().reset_index()
df_2['repo'] = 'floris_nrel'

df_3 = pd.DataFrame(results_wfc_tools)
df_3['timestamp'] = pd.to_datetime(df_3.timestamp)
df_3 = df_3.resample('7D', on='timestamp').sum().reset_index()
df_3['repo'] = 'wfc_tools'

df = df_1.append(df_2).append(df_3)
df['timestamp'] = pd.to_datetime(df.timestamp)

In [ ]:
# Melt the data for dope plotting
df_melt = pd.melt(df,id_vars=['repo','timestamp'])
df_melt = df_melt.sort_values(['timestamp'])
df_melt.head()

In [ ]:
# Dope plots
g = sns.FacetGrid(df_melt,col='variable',col_wrap=2,hue='repo',size=4,sharey=False,aspect=2)
g.map(plt.plot,'timestamp','value')
g.add_legend()
g.set_xticklabels(rotation=45)

In [ ]:
# Weekly resample
df_melt_2 = df_melt.set_index(['repo','variable'])
df_melt_2 = df_melt_2.resample('7D',on='timestamp').mean()


df_melt_2

In [ ]:
# Reformulate as rolling
# Mix together
df_1 = pd.DataFrame(results_wisdem).fillna(0).sort_values(['timestamp'])
df_1 = df_1.set_index('timestamp').cumsum().reset_index()
df_1['repo'] = 'floris_wisdem'
df_2 = pd.DataFrame(results_nrel_floris).fillna(0).sort_values(['timestamp'])
df_2 = df_2.set_index('timestamp').cumsum().reset_index()
df_2['repo'] = 'floris_nrel'
df_3 = pd.DataFrame(results_wfc_tools).fillna(0).sort_values(['timestamp'])
df_3 = df_3.set_index('timestamp').cumsum().reset_index()
df_3['repo'] = 'wfc_tools'

df = df_1.append(df_2).append(df_3)
df['timestamp'] = pd.to_datetime(df.timestamp)

In [ ]:
# Melt the data for dope plotting
df_melt = pd.melt(df,id_vars=['repo','timestamp'])
df_melt = df_melt.sort_values(['timestamp'])

In [ ]:
# Dope plots
df_melt['timestamp'] = pd.to_datetime(df_melt.timestamp)
g = sns.FacetGrid(df_melt,row='variable',hue='repo',size=2,sharey=False,aspect=4)
g.map(plt.semilogy,'timestamp','value')
g.add_legend()
g.set_xticklabels(rotation=30)

In [ ]:
# Just FLORIS
# Dope plots
df_melt['timestamp'] = pd.to_datetime(df_melt.timestamp)
df_melt_sub = df_melt[df_melt.repo=='floris_nrel']
g = sns.FacetGrid(df_melt_sub,col='variable',size=2,sharey=False,aspect=4,col_wrap=2)
g.map(plt.plot,'timestamp','value')
# g.add_legend()
g.set_xticklabels(rotation=30)

In [ ]:
df_sub = df_melt[df_melt.repo=='floris_nrel']
df_sub = df_sub[df_sub.variable.isin(['clones_total','clones_uniques'])]
df_sub

In [ ]:
# Dope plots
df_melt['timestamp'] = pd.to_datetime(df_melt.timestamp)
df_sub = df_melt[df_melt.repo=='floris_nrel']
df_sub = df_sub[df_sub.variable.isin(['clones_total','clones_uniques'])]
g = sns.FacetGrid(df_sub,row='variable',hue='repo',size=2,sharey=False,aspect=4)
g.map(plt.plot,'timestamp','value')
for ax in g.axes.flatten():
    ax.grid(True)
# g.add_legend()
# g.set_xticklabels(rotation=30)

## TOTAL COUNTS

In [ ]:
# Get total counts for all time recorded
totals = db.getTotals()
print(totals)

In [ ]:
totals